In [1]:
# TODO: Add code to halt processing if rate limit is hit
# TODO: Fix the function to drop targeted unfollowers so that it only tries to delete from current friends (not the list of everyone ever targeted0
# TODO: Add code to track whether an action is live or during burnin
# TODO: Decide whether to keep the keyword table
# TODO: Use ids instead of screen names
# TODO: Add something to filter based on the quality of tweet or user: (e.g., number of retweets/favorites, ratio of friends to followers, average hashtag or link use, post rate - especially during certain times, only target during the day)
# TODO: Create a bot-score metric (see Evernote)
# TODO: Create blacklist database (or .csv) that will never be targeted or followed and can be added to while script is running
# TODO: Create a metric for assessing accounts in terms of influence
# TODO: Once a good account is identified (e.g., they retweet something posting or have a high influence or low bot score) snowball requests based on their followers list

In [2]:
import sqlite3

# twitter package manual here: https://pypi.python.org/pypi/python-twitter/1.3.1, https://code.google.com/p/python-twitter/
import twitter 
import itertools
from datetime import date, datetime, timedelta
import time
import random
import sys
import os.path

In [3]:
# Configuration options
debug = False

# Set to False for burnin/testing phase. Actions output will be created but actions will not be executed (Still needs to be implemented)
production = True

### Better is to add this info
### to a text file and pull it in
# Client details
client_twitter_handle = 'XXXX'

client_consumer_key = 'XXXX'
client_consumer_secret = 'XXXXXX'
client_access_token_key = 'XXXXXX'
client_access_token_secret = 'XXXXX'


In [4]:
# Function to search tweets
def searchTweets (keyword_list, number_to_return, lat, lon, radi):
    
    if debug: print 'Entering searchTweets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    results_list = []
    
    # There may be a way to send all keywords at once, but this will keep the query simple for now
    for keyword in keyword_list:
        
        if ((lat==0) and (lon==0) and (radi==0)):
            temp_results = api.GetSearch(term=keyword, lang='en', result_type='recent', count=number_to_return,
                                         latitude=lat, longitude=lon, radius=radi)
            
        else:
            temp_results = api.GetSearch(term=keyword, lang='en', result_type='recent', count=number_to_return)
        
        for result in temp_results:
            result.keyword = keyword
            results_list.append(result)
        print 'There were ' + str(len(temp_results)) + ' results for the term "' + keyword + '" at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        
    if debug: print 'Exiting searchTweets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    return results_list

In [5]:
# Function to setup and reset DB
def dbManage(what_to_do):
        
    try:
        con
    except:
        con = sqlite3.connect('UserName/friendme.db')
        
    if what_to_do == 'drop':
        # To remove tables
        con.execute('''DROP TABLE incentive_actions''')
        con.execute('''DROP TABLE my_daily_followers''')
        con.execute('''DROP TABLE daily_keywords''')
        con.commit()
    if what_to_do == 'create':
        # Now add tables to database
        con.execute('''CREATE TABLE incentive_actions (user_name text, action text, action_datetime text, tweet_id int, keyword text, production text)''')
        con.execute('''CREATE TABLE my_daily_followers (followers text, list_date text)''')
        con.execute('''CREATE TABLE daily_keywords (keywords text, keywords_date text)''')
        recent = datetime.today() - timedelta(1)
        some_days_ago = datetime.today() - timedelta(364)
        many_days_ago = datetime.today() - timedelta(365)
        
        incentive_actions_testing_data = [('testingaccount1','Favorite', many_days_ago.strftime('%Y-%m-%d %H:%M:%S'),'1','testkeyword1','False'), 
                                          ('testingaccount2','Retweet', many_days_ago.strftime('%Y-%m-%d %H:%M:%S'),'2','testkeyword2','False'), 
                                          ('testingaccount3','Favorite', some_days_ago.strftime('%Y-%m-%d %H:%M:%S'),'3','testkeyword3','False'),  
                                          ('testingaccount4','Follow', some_days_ago.strftime('%Y-%m-%d %H:%M:%S'),'4','testkeyword1','False'),
                                          ('testingaccount5','Follow', recent.strftime('%Y-%m-%d %H:%M:%S'),'5','testkeyword2','False'),
                                          ('testingaccount6','Follow', recent.strftime('%Y-%m-%d %H:%M:%S'),'6','testkeyword3','False')]
        con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', incentive_actions_testing_data)
        
        daily_followers_testing_data = [('testingaccount1,testingaccount2', many_days_ago.strftime('%Y-%m-%d')), 
                                        ('testingaccount1,testingaccount2,testingaccount3,testingaccount4', some_days_ago.strftime('%Y-%m-%d')), 
                                        ('testingaccount1,testingaccount2,testingaccount3,testingaccount4,testingaccount5,testingaccount6', recent.strftime('%Y-%m-%d'))]
        con.executemany('INSERT INTO my_daily_followers VALUES (?,?)', daily_followers_testing_data)
        
        # NOTE: Now that the incentive actions have keywords attached, we can probably drop this table,
        # but we will give it some time to be sure
        daily_keywords_testing_data = [('keyword1,keyword2', many_days_ago.strftime('%Y-%m-%d')), 
                                       ('keyword1,keyword2,keyword3,keyword4', some_days_ago.strftime('%Y-%m-%d')), 
                                       ('keyword1,keyword2,keyword3,keyword4,keyword5,keyword6', recent.strftime('%Y-%m-%d'))]
        con.executemany('INSERT INTO daily_keywords VALUES (?,?)', daily_keywords_testing_data)
        
        con.commit()
    
    con.close()

In [6]:
def cleanTweets(tweets, current_followers, current_friends, targeting_delay):
    
    if debug: print 'Entering cleanTweets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        
    recent_targets = getRecentTargets(targeting_delay)    
        
    # Use tweet text to get rid of RTs and bad words
    cleaned_tweets = []
    retweet_count = 0
    bad_language_count = 0
    over_hash_count = 0
    possibly_sensitive_count = 0
    already_associated_count = 0
    
    do_not_target = set([client_twitter_handle] + current_followers + current_friends + recent_targets) 
    for t in tweets:
        drop_tweet = False
        # Get rid of retweets
        if "RT" in t.text[0:2]:
            drop_tweet = True
            retweet_count += 1
            
        # Get rid of bad words
        if any([x in t.text for x in [" cunt ", " fuck ", " sex ", " milf ", " incest ", 
                                      " anal ", " pussy ", " dick ", " ass ", "cunt", 
                                      "fuck", "#fuck", "#milf", "#sex", "#ass", "#erotic", 
                                      "#grope", " grope ", "erotic", "God", "Allah", "Jesus", 
                                      "savior", "Bible"]]):
            drop_tweet = True
            bad_language_count += 1
            
        # Get rid of tweeters that overhash
        if (t.text.count("#") >= 4):
            drop_tweet = True
            over_hash_count += 1
            
        if t.possibly_sensitive == 1:
            drop_tweet = True
            possibly_sensitive_count += 1

        # Drop owned tweets, current followers, or recent targets
        if any([x in t.user.screen_name for x in do_not_target]):
            drop_tweet = True
            already_associated_count += 1
                    
        if not drop_tweet:
            cleaned_tweets.append(t)
    
    print str(len(tweets)-len(cleaned_tweets)) + ' were dropped because they were retweets, were offensive or users were already friends or followers.'
    print str(retweet_count) + ' were identified as retweets'
    print str(bad_language_count) + ' were identified as offensive'
    print str(over_hash_count) + ' were identified as over-hashed'
    print str(possibly_sensitive_count) + ' were identified as possibly sensitive'
    print str(already_associated_count) + ' were identified as already connected or recently targeted'
    
    if debug: print 'Exiting cleanTweets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    return cleaned_tweets

In [7]:
def getRecentTargets(delay):
    
    if debug: print 'Entering getRecentTargets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Can be useful for getting the list of recent targets, so they aren't targeted again too soon
    earliest_action_date = date.today() - timedelta(delay)
    temp_recent_targets = []
    for temp_recent_target in con.execute('SELECT user_name FROM incentive_actions WHERE action_datetime > ?', [earliest_action_date.strftime('%Y-%m-%d %H:%M:%S')]):
        temp_recent_targets.append(temp_recent_target[0]) # TOFIX: This should be a named reference, not positional

    if debug: print 'Exiting getRecentTargets function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        
    return temp_recent_targets

In [8]:
#def cleanTargets(tweets, current_followers):
#    # Deduplicate remaining usernames
#    target_usernames = set([t.user.screen_name for t in tweets])#
#
#    current_followers = set(current_followers)
#    targets = target_usernames.difference(current_followers)
#    
#    # TODO: Add something to remove cases that have been recently targeted
#
#    return targets

In [9]:
def newFollowers(current_followers, number_of_lookback_days):
    
    if debug: print 'Entering newFollwers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        
    try:
        # Get previous day's friend list for comparison with today's list
        comparison_date = date.today() - timedelta(number_of_lookback_days)
        comparison_date_followers = con.execute('SELECT * FROM my_daily_followers WHERE list_date = ?', [comparison_date.strftime('%Y-%m-%d')])
        for cd_follower in comparison_date_followers:
            comparison_list = set(cd_follower[0].split(','))
        
        cf = set(current_followers)
    
        nf = cf.difference(comparison_list)
        
        print "There were " + str(len(nf)) + " new followers in the last " + str(number_of_lookback_days) + " day(s)."
    
    except:
        print 'There was a problem returning the number of new followers. Maybe the lookback date was too long or too short?
        
    if debug: print 'Exiting newFollowers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [10]:
def dropTargetedUnfollowers(current_followers, current_friends, days_since_targeted, purge, add_random):
    
    if debug: print 'Entering dropTargetedUnfollowers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        
        end_date = date.today() - timedelta(days_since_targeted)
        if (purge == True): # Remove all unfollowers targeted prior to a given date. May take a long time for large targeted lists.
            # Get previous day's friend list for comparison with today's list
            comparison_date_targets = con.execute('SELECT user_name FROM incentive_actions WHERE production = "True" AND action_datetime < ?', [end_date.strftime('%Y-%m-%d')])
        
        else: # Remove all unfollowers targeted on a specific date. Faster, but not as complete as purging unless run every day.
            # Get previous day's friend list for comparison with today's list
            comparison_date_targets = con.execute('SELECT user_name FROM incentive_actions WHERE production = "True" AND action_datetime > ? AND action_datetime < ?', [end_date.strftime('%Y-%m-%d 00:00:00'), end_date.strftime('%Y-%m-%d 23:59:59')])
 
        
        comparison_date_list = []
        for target in comparison_date_targets:
            comparison_date_list.append(target[0])
        
        cf = set(current_followers)
        
        comparison_date_set = set(comparison_date_list)
    
        unfollowers = comparison_date_set.difference(cf)
        
        print str(len(unfollowers)) + ' will be deleted.'
        
        for unfollower in unfollowers:
            try:
                api.DestroyFriendship(screen_name=unfollower)
            except:
                continue
                
        if (add_random):
            friends_range = range(1,len(current_friends))
    
            random_friends_to_drop = random.shuffle(friends_range)[0:(floor(len(current_friends)/20))]

            for random_friend in random_friends_to_drop:
                try:
                    api.DestroyFriendship(screen_name=random_friend)
                except:
                    continue

    except:
        print 'There was a problem deleting targeted unfollowers.' 
        
    if debug: print 'Exiting dropTargetedUnfollowers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [11]:
def saveCurrentFollowers(current_followers):
    
    if debug: print 'Entering saveCurrentFollowers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        current_followers_sql = [(",".join(current_followers)), (date.today().strftime('%Y-%m-%d'))]
        con.execute('INSERT INTO my_daily_followers VALUES (?,?)', current_followers_sql)
        con.commit()
        
        print "Writing today's current followers to the database was successful."
    
    except:
        print "Writing today's current followers to the database was NOT successful."
        
    if debug: print 'Exiting saveCurrentFollowers function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [12]:
def saveKeywords(daily_keywords):
    
    if debug: print 'Entering saveKeywords function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        daily_keywords_sql = [(",".join(daily_keywords)), (date.today().strftime('%Y-%m-%d'))]
        con.execute('INSERT INTO daily_keywords VALUES (?,?)', daily_keywords_sql)
        con.commit()
        
        print "Writing today's keywords to the database was successful."
    
    except:
        print "Writing today's keywords to the database was NOT successful."
        
    if debug: print 'Exiting saveKeywords function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [13]:
def fav(tweetsToFav):
    
    if debug: print 'Entering fav function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Favorites tweets that match a certain phrase (hashtag, word, etc.)
    daily_favorite_data = []
    for tweet in tweetsToFav:
        try:
            if production: result = api.CreateFavorite(id=tweet.id)
            daily_favorite_data.append((tweet.user.screen_name, 'Favorite', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("favorited: %s" % (tweet.text.encode("utf-8")))

        # when you have already favorited a tweet, this error is thrown
        except:
            print("A favoriting action didn't work for some reason")
            
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_favorite_data)
    con.commit()
    
    if debug: print 'Exiting fav function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
def rt(tweetsToRT):
    
    if debug: print 'Entering rt function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Retweets tweets that match a certain phrase (hashtag, word, etc.)
    daily_retweet_data = []
    for tweet in tweetsToRT:
        try:
            if production: result = api.PostRetweet(original_id=tweet.id)
            daily_retweet_data.append((tweet.user.screen_name, 'Retweet', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("retweeted: %s" % (tweet.text.encode("utf-8")))

        # when you have already retweeted a tweet, this error is thrown
        except:
            print("A retweeting action didn't work for some reason")
    
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_retweet_data)
    con.commit()
    
    if debug: print 'Exiting rt function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
def rtandfav(tweetsToRTandFav):
    
    if debug: print 'Entering rtandfav function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

    # Retweets and favorite tweets that match a certain phrase (hashtag, word, etc.)
    daily_retweet_and_fav_data = []
    for tweet in tweetsToRTandFav:
        try:
            if production: result = api.CreateFavorite(id=tweet.id)
            if production: result = api.PostRetweet(original_id=tweet.id)
            daily_retweet_and_fav_data.append((tweet.user.screen_name, 'Retweet and favorite', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("retweeted and favorited: %s" % (tweet.text.encode("utf-8")))

        except:
            print("A retweeting and favoriting action didn't work for some reason")
    
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_retweet_and_fav_data)
    con.commit()
    
    if debug: print 'Exiting rtandfav function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
def favandfol(tweetsToFavandFol):
    
    if debug: print 'Entering favandfol function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

    # Favorites tweets that match a certain phrase (hashtag, word, etc.)
    daily_favorite_and_fol_data = []
    for tweet in tweetsToFavandFol:
        try:
            if production: api.CreateFriendship(user_id=tweet.user.id, follow=True)
            if production: api.CreateFavorite(id=tweet.id)
            daily_favorite_and_fol_data.append((tweet.user.screen_name, 'Favorite and follow', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("favorited and followed: %s" % (tweet.text.encode("utf-8")))

        # when you have already favorited a tweet, this error is thrown
        except:
            print("A favoriting and following action didn't work for some reason")
            
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_favorite_and_fol_data)
    con.commit()
    
    if debug: print 'Entering favandfol function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [14]:
def rtandfol(tweetsToRTandFol):
    
    if debug: print 'Entering rtandfol function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Retweets tweets that match a certain phrase (hashtag, word, etc.)
    daily_retweet_and_fol_data = []
    for tweet in tweetsToRTandFol:
        try:
            if production: api.CreateFriendship(user_id=tweet.user.id, follow=True)
            if production: api.PostRetweet(original_id=tweet.id)
            daily_retweet_and_fol_data.append((tweet.user.screen_name, 'Retweet and follow', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            
            if ((debug==True) | (production == False)): print("retweeted and followed: %s" % (tweet.text.encode("utf-8")))

        # when you have already retweeted a tweet, this error is thrown
        except:
            print("A retweeting and following action didn't work for some reason")
    
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_retweet_and_fol_data)
    con.commit()
    
    if debug: print 'Exiting rtandfol function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
def skipped(tweetsToSkip):
    
    if debug: print 'Entering skipped function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Skip tweets to create a control group
    daily_skip_data = []
    for tweet in tweetsToSkip:
        try:
            daily_skip_data.append((tweet.user.screen_name, 'Skipped', datetime.today().strftime('%Y-%m-%d %H:%M:%S'),tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("skipped: %s" % (tweet.text.encode("utf-8")))

        except:
            print("A skipping action didn't work for some reason")
    
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_skip_data)
    con.commit()
    
    if debug: print 'Exiting skipped function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')


def follow(tweetersToFollow):
    
    if debug: print 'Entering follow function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    # Follows anyone who tweets about a specific phrase (hashtag, word, etc.)
    daily_follow_data = []
    for tweet in tweetersToFollow:
        try:
            if production: api.CreateFriendship(user_id=tweet.user.id, follow=True)
            daily_follow_data.append((tweet.user.screen_name, 'Follow', datetime.today().strftime('%Y-%m-%d %H:%M:%S'), tweet.id, tweet.keyword, production))
            if ((debug==True) | (production == False)): print("followed %s" % (tweet.user.screen_name))

        except TwitterHTTPError as e:
            print("A following action didn't work for some reason")
                
    con.executemany('INSERT INTO incentive_actions VALUES (?,?,?,?,?,?)', daily_follow_data)
    con.commit()
    
    if debug: print 'Exiting skipped function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [15]:
def doAction(cleaned_tweets_full, number_of_tweets_to_process_per_iter, max_iters, delay_time):
    
    if debug: print 'Entering doAction function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    print str(len(cleaned_tweets_full)) + ' cleaned tweets were sent to the doAction function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    iters = 0

    while ((len(cleaned_tweets_full) > number_of_tweets_to_process_per_iter) & (iters <= max_iters)):
        
        recent_targets = getRecentTargets(1)
        
        recleaned_tweets_full = []
        do_not_target = recent_targets
        for t in cleaned_tweets_full:
            drop_tweet = False
            # Drop owned tweets, current followers, or recent targets
            if any([x in t.user.screen_name for x in do_not_target]):
                drop_tweet = True
                        
            if not drop_tweet:
                recleaned_tweets_full.append(t)
    
        random.shuffle(recleaned_tweets_full)
        chosen_tweets = recleaned_tweets_full[0:number_of_tweets_to_process_per_iter]
        cleaned_tweets_full = recleaned_tweets_full[number_of_tweets_to_process_per_iter:]

        number_of_actions = 3
        
        action_ids = range(1,number_of_actions+1)
    
        random.shuffle(action_ids)
        
        if (action_ids[0] == 1):
            skipped(chosen_tweets)
        elif (action_ids[0] == 2):
            favandfol(chosen_tweets)
        elif (action_ids[0] == 3):
            follow(chosen_tweets)
        
        # Remove retweeting until it's clear the language filter is working well
        # elif (action_ids[0] == 4):
        #    rtandfol(chosen_tweets)
        # elif (action_ids[0] == 5):
        #    rtandfav(chosen_tweets)

        iters += 1
        
        # rate_limit_status = api.GetRateLimitStatus()
        
        # print rate_limit_status
        
        random_delay_time = range(delay_time-15, delay_time+15)
        
        random.shuffle(random_delay_time)
        
        time.sleep(random_delay_time[0])
        
    if debug: print 'Exiting doAction function at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

In [16]:
con = sqlite3.connect('UserName/friendme.db')

In [17]:
# con.execute('DELETE FROM incentive_actions WHERE user_name = "_Ayudos"')
# con.commit()

In [18]:
# dbManage('drop')

In [19]:
# dbManage('create')

In [20]:
api = twitter.Api(consumer_key=client_consumer_key,
                      consumer_secret=client_consumer_secret,
                      access_token_key=client_access_token_key,
                      access_token_secret=client_access_token_secret)

In [ ]:
while (os.path.isfile("UserName/FriendMe_Go.txt")):
    
    if debug: print 'Entering while loop 1 at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

    run_startup_stuff = True
    
    # Related to routing output to file
    orig_stdout = sys.stdout
    
    # Set main loop to run between 8am EST and 11pm EST (while not in DST)
    # TODO: Set this to take into account DST maybe? May not be a big deal
    est_time = datetime.now() - timedelta(hours=4)
    start_time = est_time.replace(hour=8, minute=0, second=0, microsecond=0)
    
    end_time = est_time.replace(hour=20, minute=30, second=0, microsecond=0) 

    while ((est_time > start_time) & (est_time < end_time) & (os.path.isfile("UserName/FriendMe_Go.txt"))):
        
        if debug: print 'Entering while loop 2 at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        
        filename="UserName/keywords.txt"
        textfile=open(filename,"r")
        inputString=textfile.read()
        keywords=inputString.split(",")
    
        while (run_startup_stuff == True):
            
            if debug: print 'Entering while loop 3 at ' + datetime.today().strftime('%Y-%m-%d %H:%M:%S')

            # Route output to file
            fi = file('UserName/FriendMe_Output_'+est_time.strftime('%Y-%m-%d')+'.txt', 'w')
            sys.stdout = fi
    
            # These two pieces will be problematic with lots of friends/followers
            # because results are returned in pages and each page is a hit on the rate limit
            # For very large numbers of friends/followers a separate function that updates
            # the database continually may be needed
            
            cur_fri = []
            my_friends = api.GetFriends(count=400) # Changing these to GetXXXIDs will allow up to 5000 results per call, but will require a lot of code reworking
            for fri in my_friends:
                cur_fri.append(fri.screen_name)
                
            cur_fol = []
            my_followers = api.GetFollowers(count=400)
            for fol in my_followers:
                cur_fol.append(fol.screen_name)
                
            saveCurrentFollowers(cur_fol)
                    
            newFollowers(cur_fol, 1)
            
            saveKeywords(keywords)
            
            run_startup_stuff = False
        
        theTweets = searchTweets(keywords, 100, 1, 1, 1)
        
        clean_tweets = cleanTweets(theTweets, cur_fol, cur_fri, 4)
        
        user = api.GetUser(screen_name = client_twitter_handle)
        
        # The number of tweets to process and time delay need to be set appropriately to 
        # avoid rate limit issues
        if (user.friends_count < 1900 | (user.friends_count < user.followers_count)): #TODO: The unfollowing function needs to be updated to include untargeted unfollwers
            doAction(clean_tweets, 1, 60, 30)
        
        est_time = datetime.now() - timedelta(hours=4)
        
        run_nightly_stuff = True

    if run_nightly_stuff == True:
        dropTargetedUnfollowers(cur_fol, cur_fri, 2, False, True)
        
        run_nightly_stuff = False
    
    if (sys.stdout != orig_stdout):
       # Route output back to original output
       sys.stdout = orig_stdout
       fi.close()
            
    # add something to send a daily email of the FriendMe_out.file
    
    # Check whether it is time to restart every 30 mins at most   
    time.sleep(1800)